In [37]:
# !pip install ".inputautocorrect/autocorrect-2.6.1.tar"
# !pip install ".inputpyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

In [38]:
# nltk.download("punkt")

In [39]:
from typing import List
import numpy as np
import pandas as pd
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
from autocorrect import Speller
from spellchecker import SpellChecker
import lightgbm as lgb
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()

In [40]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True # load seed
    
seed_everything(seed=42)

## Class CFG

In [41]:
class CFG:
    model_name="debertav3base"
    learning_rate=0.000016
    weight_decay=0.03
    hidden_dropout_prob=0.007
    attention_probs_dropout_prob=0.007
    num_train_epochs=5
    n_splits=4
    batch_size= 128
    random_seed=42
    save_steps=100
    max_length= 512
    number_base_model = 0
    test_mode = False
    device = 'CPU'
    infer_mode = True
    list_model_infer = []
weight_for_model = {'upload_model/debertav3base_lr17e-05': 0.0019079150150938945,
 'upload_model/debertav3base_lr21e-05': 0.03981389608805265,
 'upload_model/debertav3base_lr22e-05': 0.3286661462956012,
 'upload_model/debertav3large_lr12e-05': 0.7145523014565327,
 'upload_model/debertav3large_lr13e-05': 0.07855318181918133,
 'debertav3large_lr11e-05_att_0007': 0.6653541871957587,
 'debertav3large_lr17e-05_att_0007': 0.8419912793363682,
 'debertav3large_lr7e-06_att_0006': 0.9574590119255362,
 'debertav3large_lr8e-06_att_0006': 0.6382262893087756,
 'debertav3large_lr12e-05_clean_text': -0.39936047178908896,
 'debertav3large_lr7e-06_clean_text': 0.4246156134306383,
 'debertav3large_lr8e-06_clean_text': 0.7440852485992964}

scale = np.sum(list(weight_for_model.values()))
CFG.list_model_infer = weight_for_model.keys()
print("scale : " , scale)
print("list_model_infer : " , CFG.list_model_infer)

scale :  5.035864598681746
list_model_infer :  dict_keys(['upload_model/debertav3base_lr17e-05', 'upload_model/debertav3base_lr21e-05', 'upload_model/debertav3base_lr22e-05', 'upload_model/debertav3large_lr12e-05', 'upload_model/debertav3large_lr13e-05', 'debertav3large_lr11e-05_att_0007', 'debertav3large_lr17e-05_att_0007', 'debertav3large_lr7e-06_att_0006', 'debertav3large_lr8e-06_att_0006', 'debertav3large_lr12e-05_clean_text', 'debertav3large_lr7e-06_clean_text', 'debertav3large_lr8e-06_clean_text'])


In [42]:
# print device
if CFG.device != 'CPU':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # print device 
else :
    device = torch.device("cpu")
print(device)

cpu


## Dataload

In [43]:
DATA_DIR = "input/commonlit-evaluate-student-summaries/"
prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

## Exploratory Data Analysis

In [44]:
# prompts_train.head()

## Preprocess 2


In [45]:
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import ne_chunk, word_tokenize, pos_tag
from bs4 import BeautifulSoup

# nltk.downloader.download('vader_lexicon')
import pyphen
from nltk.sentiment import SentimentIntensityAnalyzer

dic = pyphen.Pyphen(lang='en')
sid = SentimentIntensityAnalyzer()

class Preprocessor:
    def __init__(self, 
                model_name: str,
                clean_text_mode = False) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(f"input/{model_name}")
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker() 
        self.clean_text_mode = clean_text_mode
        
    def calculate_text_similarity(self, row):
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([row['prompt_text'], row['text']])
        return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2]).flatten()[0]
    
    def sentiment_analysis(self, text):
        analysis = TextBlob(text)
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity
    
    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss
    
    def calculate_unique_words(self,text):
        unique_words = set(text.split())
        return len(unique_words)
    
    def add_spelling_dictionary(self, tokens: List[str]) -> List[str]:
        """dictionary update for pyspell checker and autocorrect"""
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})
        
    def calculate_pos_ratios(self , text):
        pos_tags = pos_tag(nltk.word_tokenize(text))
        pos_counts = Counter(tag for word, tag in pos_tags)
        total_words = len(pos_tags)
        ratios = {tag: count / total_words for tag, count in pos_counts.items()}
        return ratios
    
    def calculate_punctuation_ratios(self,text):
        total_chars = len(text)
        punctuation_counts = Counter(char for char in text if char in '.,!?;:"()[]{}')
        ratios = {char: count / total_chars for char, count in punctuation_counts.items()}
        return ratios
    
    def calculate_keyword_density(self,row):
        keywords = set(row['prompt_text'].split())
        text_words = row['text'].split()
        keyword_count = sum(1 for word in text_words if word in keywords)
        return keyword_count / len(text_words)
    
    def count_syllables(self,word):
        hyphenated_word = dic.inserted(word)
        return len(hyphenated_word.split('-'))

    def flesch_reading_ease_manual(self,text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        total_syllables = sum(self.count_syllables(word) for word in TextBlob(text).words)

        if total_sentences == 0 or total_words == 0:
            return 0

        flesch_score = 206.835 - 1.015 * (total_words / total_sentences) - 84.6 * (total_syllables / total_words)
        return flesch_score
    
    def flesch_kincaid_grade_level(self, text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        total_syllables = sum(self.count_syllables(word) for word in TextBlob(text).words)

        if total_sentences == 0 or total_words == 0:
            return 0

        fk_grade = 0.39 * (total_words / total_sentences) + 11.8 * (total_syllables / total_words) - 15.59
        return fk_grade
    
    def gunning_fog(self, text):
        total_sentences = len(TextBlob(text).sentences)
        total_words = len(TextBlob(text).words)
        complex_words = sum(1 for word in TextBlob(text).words if self.count_syllables(word) > 2)

        if total_sentences == 0 or total_words == 0:
            return 0

        fog_index = 0.4 * ((total_words / total_sentences) + 100 * (complex_words / total_words))
        return fog_index
    
    def calculate_sentiment_scores(self,text):
        sentiment_scores = sid.polarity_scores(text)
        return sentiment_scores
    
    def count_difficult_words(self, text, syllable_threshold=3):
        words = TextBlob(text).words
        difficult_words_count = sum(1 for word in words if self.count_syllables(word) >= syllable_threshold)
        return difficult_words_count

    def text_cleaning(self, text):
        '''
        Cleans text into a basic form for NLP. Operations include the following:-
        1. Remove special charecters like &, #, etc
        2. Removes extra spaces
        3. Removes embedded URL links
        4. Removes HTML tags
        5. Removes emojis

        text - Text piece to be cleaned.
        '''
        template = re.compile(r'https?://\S+|www\.\S+')  # Removes website links
        text = template.sub(r'', text)

        soup = BeautifulSoup(text, 'lxml')  # Removes HTML tags
        only_text = soup.get_text()
        text = only_text

        emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)

        text = re.sub(r"[^a-zA-Z\d]", " ", text) # Remove special Charecters
        text = re.sub('\n+', '\n', text) 
        text = re.sub('\.+', '.', text) 
        text = re.sub(' +', ' ', text) # Remove Extra Spaces 

        return text
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        # experiment
        if self.clean_text_mode :
            summaries['text'] = summaries['text'].progress_apply(self.text_cleaning)
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(word_tokenize(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: word_tokenize(x)
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(word_tokenize(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: word_tokenize(x)
        )
        
        # Add prompt tokens into spelling checker dictionary
        prompts["prompt_tokens"].apply(
            lambda x: self.add_spelling_dictionary(x)
        )
        
        prompts['gunning_fog_prompt'] = prompts['prompt_text'].apply(self.gunning_fog)
        prompts['flesch_kincaid_grade_level_prompt'] = prompts['prompt_text'].apply(self.flesch_kincaid_grade_level)
        prompts['flesch_reading_ease_prompt'] = prompts['prompt_text'].apply(self.flesch_reading_ease_manual)

        
#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        if self.clean_text_mode:
            summaries["fixed_summary_text"] = summaries["text"].progress_apply(
                lambda x: self.text_cleaning(x)
            )
            summaries["fixed_summary_text"] = summaries["text"].progress_apply(
                lambda x: self.speller(x)
            )
        else:
            summaries["fixed_summary_text"] = summaries["text"].progress_apply(
                lambda x: self.speller(x)
            )
        
        
        # count misspelling
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)
        
        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")
        input_df['flesch_reading_ease'] = input_df['text'].apply(self.flesch_reading_ease_manual)
        input_df['word_count'] = input_df['text'].apply(lambda x: len(x.split()))
        input_df['sentence_length'] = input_df['text'].apply(lambda x: len(x.split('.')))
        input_df['vocabulary_richness'] = input_df['text'].apply(lambda x: len(set(x.split())))

        input_df['word_count2'] = [len(t.split(' ')) for t in input_df.text]
        input_df['num_unq_words']=[len(list(set(x.lower().split(' ')))) for x in input_df.text]
        input_df['num_chars']= [len(x) for x in input_df.text]

        # Additional features
        input_df['avg_word_length'] = input_df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]))
        input_df['comma_count'] = input_df['text'].apply(lambda x: x.count(','))
        input_df['semicolon_count'] = input_df['text'].apply(lambda x: x.count(';'))

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)
        
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)
        
        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)
        
        input_df['exclamation_count'] = input_df['text'].apply(lambda x: x.count('!'))
        input_df['question_count'] = input_df['text'].apply(lambda x: x.count('?'))
        input_df['pos_ratios'] = input_df['text'].apply(self.calculate_pos_ratios)

        # Convert the dictionary of POS ratios into a single value (mean)
        input_df['pos_mean'] = input_df['pos_ratios'].apply(lambda x: np.mean(list(x.values())))
        input_df['punctuation_ratios'] = input_df['text'].apply(self.calculate_punctuation_ratios)

        # Convert the dictionary of punctuation ratios into a single value (sum)
        input_df['punctuation_sum'] = input_df['punctuation_ratios'].apply(lambda x: np.sum(list(x.values())))
        input_df['keyword_density'] = input_df.apply(self.calculate_keyword_density, axis=1)
        input_df['jaccard_similarity'] = input_df.apply(lambda row: len(set(word_tokenize(row['prompt_text'])) & set(word_tokenize(row['text']))) / len(set(word_tokenize(row['prompt_text'])) | set(word_tokenize(row['text']))), axis=1)
        tqdm.pandas(desc="Performing Sentiment Analysis")
        input_df[['sentiment_polarity', 'sentiment_subjectivity']] = input_df['text'].progress_apply(
            lambda x: pd.Series(self.sentiment_analysis(x))
        )
        tqdm.pandas(desc="Calculating Text Similarity")
        input_df['text_similarity'] = input_df.progress_apply(self.calculate_text_similarity, axis=1)
        #Calculate sentiment scores for each row
        input_df['sentiment_scores'] = input_df['text'].apply(self.calculate_sentiment_scores)
        
        input_df['gunning_fog'] = input_df['text'].apply(self.gunning_fog)
        input_df['flesch_kincaid_grade_level'] = input_df['text'].apply(self.flesch_kincaid_grade_level)
        input_df['count_difficult_words'] = input_df['text'].apply(self.count_difficult_words)

        # Convert sentiment_scores into individual columns
        sentiment_columns = pd.DataFrame(list(input_df['sentiment_scores']))
        input_df = pd.concat([input_df, sentiment_columns], axis=1)
        input_df['sentiment_scores_prompt'] = input_df['prompt_text'].apply(self.calculate_sentiment_scores)
        # Convert sentiment_scores_prompt into individual columns
        sentiment_columns_prompt = pd.DataFrame(list(input_df['sentiment_scores_prompt']))
        sentiment_columns_prompt.columns = [col +'_prompt' for col in sentiment_columns_prompt.columns]
        input_df = pd.concat([input_df, sentiment_columns_prompt], axis=1)
        columns =  ['pos_ratios', 'sentiment_scores', 'punctuation_ratios', 'sentiment_scores_prompt']
        cols_to_drop = [col for col in columns if col in input_df.columns]
        if cols_to_drop:
            input_df = input_df.drop(columns=cols_to_drop)
        
        print(cols_to_drop)
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    

## Create the train and test sets


In [46]:
if CFG.test_mode : 
    prompts_train = prompts_train[:12]
    prompts_test = prompts_test[:12]
    summaries_train = summaries_train[:12]
    summaries_test = summaries_test[:12]

## Load the train data

In [47]:
preprocessor = Preprocessor(model_name=CFG.model_name)
# train = preprocessor.run(prompts_train, summaries_train, mode="train")
test = preprocessor.run(prompts_test, summaries_test, mode="test")

# load preprocess
preprocessor = Preprocessor(model_name=CFG.model_name, clean_text_mode=True)
test_2 = preprocessor.run(prompts_test, summaries_test, mode="test")# save data train to csv 
# train_2 = preprocessor.run(prompts_train, summaries_train, mode="train")
# train.to_csv("input/train_clean_text.csv", index=False)
# load train data
# train = pd.read_csv("input/train_clean_text.csv")
# train_2.to_csv("input/train_preprocess_2.csv", index=False)
train = pd.read_csv("input/train.csv")
train_2 = pd.read_csv("input/train_preprocess_2.csv")

Calculating Text Similarity: 100%|██████████| 4/4 [00:00<00:00, 830.35it/s]


['pos_ratios', 'sentiment_scores', 'punctuation_ratios', 'sentiment_scores_prompt']


Calculating Text Similarity: 100%|██████████| 4/4 [00:00<00:00, 857.86it/s]


['pos_ratios', 'sentiment_scores', 'punctuation_ratios', 'sentiment_scores_prompt']


In [48]:
gkf = GroupKFold(n_splits=CFG.n_splits)

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i
    train_2.loc[val_index, "fold"] = i
train.head(3)

,student_id,prompt_id,text,content,wording,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,...,neu,pos,compound,neg_prompt,neu_prompt,pos_prompt,compound_prompt,fold,wording_pred,content_pred
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,64,The third wave was an experimental see how peo...,5,Summarize how the Third Wave developed over su...,The Third Wave,...,0.832,0.135,0.7845,0.027,0.873,0.100,0.9915,3.0,-0.222891,0.84066
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,54,They would rub it up with soda to make the sme...,2,Summarize the various ways the factory would u...,Excerpt from The Jungle,...,0.946,0.054,0.4310,0.086,0.879,0.035,-0.9949,2.0,-0.800661,-0.27018
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,269,"In Egypt, there were many occupations and soci...",32,"In complete sentences, summarize the structure...",Egyptian Social Structure,...,0.814,0.139,0.9725,0.063,0.845,0.092,0.9283,1.0,2.513938,2.13275


## Model Function Definition

In [49]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

## Deberta Regressor

In [50]:
class ScoreRegressor:
    def __init__(self, 
                model_name: str,
                model_dir: str,
                target: list,
                hidden_dropout_prob: float,
                attention_probs_dropout_prob: float,
                max_length: int,
                ):
        self.inputs = ["prompt_text", "prompt_title", "prompt_question", "fixed_summary_text"] # fix summary text have prompt text in it 
        self.input_col = "input"
        
        self.text_cols = [self.input_col] 
        self.target = target
        self.target_cols = target

        self.model_name = model_name
        lr = str(CFG.learning_rate).replace(".", "")
        self.model_dir = model_dir
        self.max_length = max_length
        
        self.tokenizer = AutoTokenizer.from_pretrained(f"input/{model_name}")
        self.model_config = AutoConfig.from_pretrained(f"input/{model_name}" )
        # print(self.model_config)
        self.model_config.update({
            "hidden_dropout_prob": hidden_dropout_prob,
            "attention_probs_dropout_prob": attention_probs_dropout_prob,
            "num_labels": 2,
            "problem_type": "regression",
        })
        seed_everything(seed=42)

        self.data_collator = DataCollatorWithPadding(
            tokenizer=self.tokenizer
        )


    def tokenize_function(self, examples: pd.DataFrame):
        # labels = ['content' , 'wording']
        # print('labels', labels)
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return {
            **tokenized,
            "labels": [examples['content'], examples['wording']],
        }
    
    def tokenize_function_test(self, examples: pd.DataFrame):
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return tokenized
        
    def predict(self, 
                test_df: pd.DataFrame,
                fold: int,
               ):
        """predict content score"""
        
        sep = self.tokenizer.sep_token
        in_text = (
                    test_df["prompt_title"] + sep 
                    + test_df["prompt_question"] + sep 
                    + test_df["fixed_summary_text"]
                  )
        test_df[self.input_col] = in_text

        test_ = test_df[[self.input_col]]
    
        test_dataset = Dataset.from_pandas(test_, preserve_index=False) 
        test_tokenized_dataset = test_dataset.map(self.tokenize_function_test, batched=True)

        model_content = AutoModelForSequenceClassification.from_pretrained(f"{self.model_dir}")
        model_content.eval()
        
        # eg. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold)) 
        # print("model_fold_dir",model_fold_dir)
        test_args = TrainingArguments(
            output_dir=model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = CFG.batch_size,   
            dataloader_drop_last = False,
        )

        # init trainer
        infer_content = Trainer(
                      model = model_content, 
                      tokenizer=self.tokenizer,
                      data_collator=self.data_collator,
                      args = test_args)

        preds = infer_content.predict(test_tokenized_dataset)[0]

        return preds

## Train by fold function


In [51]:
def validate(
    train_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    model_dir_base: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ) -> pd.DataFrame:
    """predict oof data"""
    for fold in range(CFG.n_splits):
        # print(f"fold {fold}:")
        
        valid_data = train_df[train_df["fold"] == fold]
        
        if save_each_model == True:
            model_dir =  f"{target}/{model_dir_base}/fold_{fold}"
        else: 
            model_dir =  f"{model_dir_base}/fold_{fold}"
        csr = ScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=valid_data, 
            fold=fold
        )
        # print('pred shape', pred.shape)
        train_df.loc[valid_data.index, f"wording_pred"] = pred[:,0]
        train_df.loc[valid_data.index, f"content_pred"] = pred[:,1]

    return train_df
    
def predict(
    test_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    model_dir_base: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ):
    """predict using mean folds"""
    for fold in range(CFG.n_splits):
        # print(f"fold {fold}:")
        
        if save_each_model == True:
            model_dir =  f"{target}/{model_dir_base}/fold_{fold}"
        else: 
            model_dir =  f"{model_dir_base}/fold_{fold}"
        csr = ScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir, 
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=test_df, 
            fold=fold
        )
        
        # test_df[f"{target}_pred_{fold}"] = pred
        test_df[f"wording_pred_{fold}"] = pred[:,0]
        test_df[f"content_pred_{fold}"] = pred[:,1]
        
    # test_df[f"{target}"] = test_df[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)
    test_df[f"wording_pred"] = test_df[[f"wording_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)
    test_df[f"content_pred"] = test_df[[f"content_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)
    return test_df
targets =  ["content", "wording"]


## Infer model dont clean text



In [52]:
# ensembling_results_test = pd.DataFrame()

In [53]:
# dem = 0
# if CFG.infer_mode:
#     for model_dir in CFG.list_model_infer:
#         if 'clean_text' in model_dir:
#             continue
#         if CFG.number_base_model > 0:
#             print("percent of model", dem/CFG.number_base_model)
#         print(model_dir)    
#         dem = dem +1 
#         if dem >= CFG.number_base_model:
#             CFG.batch_size = 16
#             CFG.max_length = 1462
#             CFG.model_name = "debertav3large"
#         # train = validate(
#         #     train,
#         #     target=targets,
#         #     save_each_model=False,
#         #     model_name=CFG.model_name,
#         #     model_dir_base = model_dir,
#         #     hidden_dropout_prob=CFG.hidden_dropout_prob,
#         #     attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
#         #     max_length=CFG.max_length
#         # )
#         # for target in targets:
#         #     rmse = mean_squared_error(train[target], train[f"{target}_pred"], squared=False)
#         #     print(f"cv {target} rmse: {rmse}")
#         # print('done validate')
#         test = predict(
#             test,
#             target=targets,
#             save_each_model=False,
#             model_name=CFG.model_name,
#             model_dir_base = model_dir,
#             hidden_dropout_prob=CFG.hidden_dropout_prob,
#             attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
#             max_length=CFG.max_length
#         )
#         print('done predict')
#         # add wording_pred and content_pred to ensembling_results
#         # ensembling_results_val[f"{model_dir}_wording_pred"] = train["wording_pred"]
#         # ensembling_results_val[f"{model_dir}_content_pred"] = train["content_pred"]
#         ensembling_results_test[f"{model_dir}_wording_pred"] = test["wording_pred"]
#         ensembling_results_test[f"{model_dir}_content_pred"] = test["content_pred"]
#         # print('ensembling_results_val \n', ensembling_results_val.head() )
        

## infer model clean_text

In [54]:
# if CFG.infer_mode:
#     for model_dir in CFG.list_model_infer:
#         if 'clean_text' not in model_dir:
#             continue
#         print(model_dir)   
#         CFG.batch_size = 5
#         # train_2 = validate(
#         #     train_2,
#         #     target=targets,
#         #     save_each_model=False,
#         #     model_name=CFG.model_name,
#         #     model_dir_base = model_dir,
#         #     hidden_dropout_prob=CFG.hidden_dropout_prob,
#         #     attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
#         #     max_length=CFG.max_length
#         # )
#         # for target in targets:
#         #     rmse = mean_squared_error(train_2[target], train_2[f"{target}_pred"], squared=False)
#         #     print(f"cv {target} rmse: {rmse}")
#         # print('done validate')
#         test_2 = predict(
#             test_2,
#             target=targets,
#             save_each_model=False,
#             model_name=CFG.model_name,
#             model_dir_base = model_dir,
#             hidden_dropout_prob=CFG.hidden_dropout_prob,
#             attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
#             max_length=CFG.max_length
#         )
#         print('done predict')
#         # add wording_pred and content_pred to ensembling_results
#         # ensembling_results_val[f"{model_dir}_wording_pred"] = train_2["wording_pred"]
#         # ensembling_results_val[f"{model_dir}_content_pred"] = train_2["content_pred"]
#         ensembling_results_test[f"{model_dir}_wording_pred"] = test_2["wording_pred"]
#         ensembling_results_test[f"{model_dir}_content_pred"] = test_2["content_pred"]
#         # print('ensembling_results_val \n', ensembling_results_val.head())

## LGBM model

In [55]:
# save ensembling_results_val and ensembling_results_test
# ensembling_results_val.to_csv("input/ensembling_results_val_clean_text_1110.csv", index=False)
# ensembling_results_test.to_csv("ensembling_results_test.csv", index=False)
# load ensembling_results_val and ensembling_results_test

In [56]:
ensembling_results_val = pd.read_csv("input/ensembling_results_val_clean_text_1110.csv")
ensembling_results_val.head(2)

,upload_model/debertav3base_lr5e-05_wording_pred,upload_model/debertav3base_lr5e-05_content_pred,upload_model/debertav3base_lr15e-05_wording_pred,upload_model/debertav3base_lr15e-05_content_pred,upload_model/debertav3base_lr17e-05_wording_pred,upload_model/debertav3base_lr17e-05_content_pred,upload_model/debertav3base_lr18e-05_wording_pred,upload_model/debertav3base_lr18e-05_content_pred,upload_model/debertav3base_lr21e-05_wording_pred,upload_model/debertav3base_lr21e-05_content_pred,...,debertav3large_lr7e-06_att_0006_wording_pred,debertav3large_lr7e-06_att_0006_content_pred,debertav3large_lr8e-06_att_0006_wording_pred,debertav3large_lr8e-06_att_0006_content_pred,debertav3large_lr12e-05_clean_text_wording_pred,debertav3large_lr12e-05_clean_text_content_pred,debertav3large_lr7e-06_clean_text_wording_pred,debertav3large_lr7e-06_clean_text_content_pred,debertav3large_lr8e-06_clean_text_wording_pred,debertav3large_lr8e-06_clean_text_content_pred
0,0.289577,0.754494,0.255495,0.757381,0.253911,0.798586,0.238143,0.805690,0.226564,0.868932,...,0.029472,0.575491,0.008973,0.591914,0.064121,0.634637,0.096928,0.661756,0.160011,0.796565
1,-0.702410,-0.433037,-0.801842,-0.464458,-0.772340,-0.413345,-0.665685,-0.327464,-0.641115,-0.371242,...,-0.972083,-0.664230,-1.004822,-0.483753,-0.819019,-0.288662,-0.934642,-0.343674,-0.887494,-0.609832


In [57]:
# replace upload_model/ with ""
# ensembling_results_val = ensembling_results_val.rename(columns=lambda x: x.replace("upload_model/", ""))

## Find the best weight with optuna

In [58]:
# weight_for_model = {}
# scale = 0 


In [59]:
# weight_for_model = {'upload_model/debertav3base_lr17e-05': -0.2566815804974416,
#  'upload_model/debertav3base_lr18e-05': 0.2377480721568757,
#  'upload_model/debertav3base_lr21e-05': 0.3056613125593628,
#  'upload_model/debertav3base_lr22e-05': 0.2365739173075792,
#  'upload_model/debertav3large_lr12e-05': 0.4367184075524858,
#  'upload_model/debertav3large_lr13e-05': 0.5280894407480876,
#  'debertav3large_lr8e-06_att_0007': 0.13148220704647767,
#  'debertav3large_lr11e-05_att_0007': 0.031119103031034406,
#  'debertav3large_lr13e-05_att_0007': 0.16499189628048969,
#  'debertav3large_lr14e-05_att_0007': 0.43258254559068815,
#  'debertav3large_lr17e-05_att_0007': 0.5705531664577692,
#  'debertav3large_lr18e-05_att_0007': -0.00013989467668473754,
#  'debertav3large_lr7e-06_att_0006': 0.6892058130214527,
#  'debertav3large_lr8e-06_att_0006': 0.90651196780627,
#  'debertav3large_lr12e-05_clean_text': -0.2307792071192416,
#  'debertav3large_lr7e-06_clean_text': 0.6074702448549276,
#  'debertav3large_lr8e-06_clean_text': 0.5278679871145304}

# scale = np.sum([weight_for_model[model] for model in CFG.list_model_infer])
# CFG.list_model_infer = weight_for_model.keys()
# CFG.list_model_infer = []
# CFG.list_model_infer_2 = []
# for key in weight_for_model.keys():
#     if "clean_text" not in key:
#         CFG.list_model_infer.append(key)
#     else:
#         CFG.list_model_infer_2.append(key)

In [60]:
for model in CFG.list_model_infer:
    results = ensembling_results_val[[f"{model}_wording_pred", f"{model}_content_pred"]]
    mcrmse = compute_mcrmse((results.values, train[targets].values))
    print(f"{model} mcrmse: {mcrmse['mcrmse']}")

upload_model/debertav3base_lr17e-05 mcrmse: 0.5543694149276929
upload_model/debertav3base_lr21e-05 mcrmse: 0.553756334918185
upload_model/debertav3base_lr22e-05 mcrmse: 0.5480178090069433
upload_model/debertav3large_lr12e-05 mcrmse: 0.5531885292652218
upload_model/debertav3large_lr13e-05 mcrmse: 0.5520867504987613
debertav3large_lr11e-05_att_0007 mcrmse: 0.5395042177968217
debertav3large_lr17e-05_att_0007 mcrmse: 0.5517737704359214
debertav3large_lr7e-06_att_0006 mcrmse: 0.5199209254995885
debertav3large_lr8e-06_att_0006 mcrmse: 0.5229320451563924
debertav3large_lr12e-05_clean_text mcrmse: 0.5873813203343701
debertav3large_lr7e-06_clean_text mcrmse: 0.5629714450649694
debertav3large_lr8e-06_clean_text mcrmse: 0.5638971562771529


In [61]:
print(weight_for_model)
print(scale)

{'upload_model/debertav3base_lr17e-05': 0.0019079150150938945, 'upload_model/debertav3base_lr21e-05': 0.03981389608805265, 'upload_model/debertav3base_lr22e-05': 0.3286661462956012, 'upload_model/debertav3large_lr12e-05': 0.7145523014565327, 'upload_model/debertav3large_lr13e-05': 0.07855318181918133, 'debertav3large_lr11e-05_att_0007': 0.6653541871957587, 'debertav3large_lr17e-05_att_0007': 0.8419912793363682, 'debertav3large_lr7e-06_att_0006': 0.9574590119255362, 'debertav3large_lr8e-06_att_0006': 0.6382262893087756, 'debertav3large_lr12e-05_clean_text': -0.39936047178908896, 'debertav3large_lr7e-06_clean_text': 0.4246156134306383, 'debertav3large_lr8e-06_clean_text': 0.7440852485992964}
5.035864598681746


In [62]:
train[f"wording_pred"] = np.sum([ensembling_results_val[f"{model}_wording_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale    
train[f"content_pred"] = np.sum([ensembling_results_val[f"{model}_content_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale
# test[f"wording_pred"] = np.sum([ensembling_results_test[f"{model}_wording_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale
# test[f"content_pred"] = np.sum([ensembling_results_test[f"{model}_content_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale


In [63]:
# save train and test
# train.to_csv("input/train.csv", index=False)
# load train and test
# test = pd.read_csv("input/test.csv")

In [64]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title", 
                "prompt_text"
               ] + targets

In [65]:
train.head(2)

,student_id,prompt_id,text,content,wording,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,...,neu,pos,compound,neg_prompt,neu_prompt,pos_prompt,compound_prompt,fold,wording_pred,content_pred
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,64,The third wave was an experimental see how peo...,5,Summarize how the Third Wave developed over su...,The Third Wave,...,0.832,0.135,0.7845,0.027,0.873,0.100,0.9915,3.0,-0.002716,0.735807
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,54,They would rub it up with soda to make the sme...,2,Summarize the various ways the factory would u...,Excerpt from The Jungle,...,0.946,0.054,0.4310,0.086,0.879,0.035,-0.9949,2.0,-0.872361,-0.496474


In [66]:
def create_model_dict(targets,train):
  model_dict = {}
  for target in targets:
      models = []

      for fold in range(CFG.n_splits):
          X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns, inplace=False)
          y_train_cv = train[train["fold"] != fold][target]

          X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
          y_eval_cv = train[train["fold"] == fold][target]

          dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
          dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

          params = {
              'boosting_type': 'gbdt',
              'random_state': 42,
              'objective': 'regression',
              'metric': 'rmse',
              'learning_rate': 0.048,
              'max_depth': 3,
              'lambda_l1': 0.0,
              'lambda_l2': 0.011,
              'verbose': -1,
          }

          evaluation_results = {}
          model = lgb.train(params,
                            num_boost_round=10000,
                            valid_names=['train', 'valid'],
                            train_set=dtrain,
                            valid_sets=dval,
                            callbacks=[
                                lgb.early_stopping(stopping_rounds=70, verbose=False),
                                # lgb.log_evaluation(100),
                                lgb.callback.record_evaluation(evaluation_results)
                              ],
                            )
          models.append(model)

      model_dict[target] = models
  return model_dict
model_dict = create_model_dict(targets,train)

## CV Score

In [67]:
# cv
import optuna
def cal_mcrmse(model_dict, targets, train  = train , print_rmse = False):
    rmses = []
    for target in targets:
        models = model_dict[target]

        preds = []
        trues = []
        
        for fold, model in enumerate(models):
            X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns , inplace=False)
            y_eval_cv = train[train["fold"] == fold][target]

            pred = model.predict(X_eval_cv)

            trues.extend(y_eval_cv)
            preds.extend(pred)
            
        rmse = np.sqrt(mean_squared_error(trues, preds))
        if print_rmse:
            print(f"{target}_rmse : {rmse}")
        rmses = rmses + [rmse]
    return sum(rmses) / len(rmses)
mcrmse = cal_mcrmse(model_dict, targets, train = train , print_rmse = True)
print(f"mcrmse: {mcrmse}")

content_rmse : 0.4110683251226793
wording_rmse : 0.5342570264645475
mcrmse: 0.47266267579361343


## Optuna

In [68]:
weight_for_model = {'upload_model/debertav3base_lr17e-05': 0.0019079150150938945,
 'upload_model/debertav3base_lr21e-05': 0.03981389608805265,
 'upload_model/debertav3base_lr22e-05': 0.3286661462956012,
 'upload_model/debertav3large_lr12e-05': 0.7145523014565327,
 'upload_model/debertav3large_lr13e-05': 0.07855318181918133,
 'debertav3large_lr11e-05_att_0007': 0.6653541871957587,
 'debertav3large_lr17e-05_att_0007': 0.8419912793363682,
 'debertav3large_lr7e-06_att_0006': 0.9574590119255362,
 'debertav3large_lr8e-06_att_0006': 0.6382262893087756,
#  'debertav3large_lr12e-05_clean_text': -0.39936047178908896,
 'debertav3large_lr7e-06_clean_text': 0.4246156134306383,
 'debertav3large_lr8e-06_clean_text': 0.7440852485992964}

scale = np.sum(list(weight_for_model.values()))
CFG.list_model_infer = weight_for_model.keys()
print("scale : " , scale)
print("list_model_infer : " , CFG.list_model_infer)

scale :  5.435225070470835
list_model_infer :  dict_keys(['upload_model/debertav3base_lr17e-05', 'upload_model/debertav3base_lr21e-05', 'upload_model/debertav3base_lr22e-05', 'upload_model/debertav3large_lr12e-05', 'upload_model/debertav3large_lr13e-05', 'debertav3large_lr11e-05_att_0007', 'debertav3large_lr17e-05_att_0007', 'debertav3large_lr7e-06_att_0006', 'debertav3large_lr8e-06_att_0006', 'debertav3large_lr7e-06_clean_text', 'debertav3large_lr8e-06_clean_text'])


In [69]:
import optuna
def objective(trial,train = train):
    weight_for_model = {}
    for model in CFG.list_model_infer:
        # if "lr12e-05_clean_text" in model:
        #     weight_for_model[model] = trial.suggest_float(model, -0.5, 0)
        # elif "debertav3base_lr17e-05" in model:
        #     weight_for_model[model] = trial.suggest_float(model, -0.5, 1)
        if "clean_text" not in model:
            # weight_for_model[model] = trial.suggest_float(model, 0, 1)
            weight_for_model[model] = trial.suggest_float(model, -0.5, 1, step=0.01)
        else:
            weight_for_model[model] = trial.suggest_float(model, -0.5, 1,step=0.01)
    scale = np.sum([weight_for_model[model] for model in CFG.list_model_infer])
    train[f"wording_pred"] = np.sum([ensembling_results_val[f"{model}_wording_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale    
    train[f"content_pred"] = np.sum([ensembling_results_val[f"{model}_content_pred"] * weight_for_model[model] for model in CFG.list_model_infer], axis=0) / scale
    model_dict = create_model_dict(targets,train)
    lost = cal_mcrmse(model_dict, targets)
    # print(f"mcrmse: {lost}")
    # print(f"weight_for_model: {weight_for_model}")
    return lost

In [70]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=70)
# print('Best trial:')
# trial_ = study.best_trial

# print('Value: ', trial_.value)


In [71]:
# trial_ = study.best_trial

# print('Value: ', trial_.value)
# for model in CFG.list_model_infer:
#     weight_for_model[model] = trial_.params[model]
# weight_for_model

In [72]:
# Value:  0.47266267579361343
# {'upload_model/debertav3base_lr17e-05': 0.0019079150150938945,
#  'upload_model/debertav3base_lr21e-05': 0.03981389608805265,
#  'upload_model/debertav3base_lr22e-05': 0.3286661462956012,
#  'upload_model/debertav3large_lr12e-05': 0.7145523014565327,
#  'upload_model/debertav3large_lr13e-05': 0.07855318181918133,
#  'debertav3large_lr11e-05_att_0007': 0.6653541871957587,
#  'debertav3large_lr17e-05_att_0007': 0.8419912793363682,
#  'debertav3large_lr7e-06_att_0006': 0.9574590119255362,
#  'debertav3large_lr8e-06_att_0006': 0.6382262893087756,
#  'debertav3large_lr12e-05_clean_text': -0.39936047178908896,
#  'debertav3large_lr7e-06_clean_text': 0.4246156134306383,
#  'debertav3large_lr8e-06_clean_text': 0.7440852485992964}
# Value:  0.47429363257870777
# {'upload_model/debertav3base_lr21e-05': 0.5169433346605325,
#  'upload_model/debertav3base_lr22e-05': 0.3160789115864947,
#  'upload_model/debertav3large_lr12e-05': 0.24532844648357505,
#  'upload_model/debertav3large_lr13e-05': 0.40061388588662983,
#  'debertav3large_lr11e-05_att_0007': 0.6918758534348053,
#  'debertav3large_lr17e-05_att_0007': 0.8988704616729967,
#  'debertav3large_lr7e-06_att_0006': 0.6345256607067367,
#  'debertav3large_lr8e-06_att_0006': 0.41687335172737405,
#  'debertav3large_lr7e-06_clean_text': 0.3243698075644693,
#  'debertav3large_lr8e-06_clean_text': 0.37001952423498624}

# Value:  0.47449842967399963
# {'upload_model/debertav3base_lr17e-05': -0.5426571999177554,
#  'upload_model/debertav3base_lr18e-05': -0.41787792532519863,
#  'upload_model/debertav3base_lr21e-05': 0.17508025799518462,
#  'upload_model/debertav3base_lr22e-05': 0.6593405312017622,
#  'upload_model/debertav3large_lr12e-05': 0.8448523828520981,
#  'upload_model/debertav3large_lr13e-05': 0.05576651155391085,
#  'debertav3large_lr8e-06_att_0007': 0.8261560233885576,
#  'debertav3large_lr13e-05_att_0007': 0.235186734542206,
#  'debertav3large_lr17e-05_att_0007': 0.8366619848207301,
#  'debertav3large_lr7e-06_att_0006': 0.6825759732490162,
#  'debertav3large_lr8e-06_att_0006': 0.7447354029009094,
#  'debertav3large_lr12e-05_clean_text': 0.001859259890705671,
#  'debertav3large_lr7e-06_clean_text': 0.6840497323404321,
#  'debertav3large_lr8e-06_clean_text': 0.5555605862244913}

In [73]:

# {'upload_model/debertav3base_lr17e-05': 0,
#  'upload_model/debertav3base_lr18e-05': 0,
#  'upload_model/debertav3base_lr21e-05': 0,
#  'upload_model/debertav3base_lr22e-05': 1,
#  'upload_model/debertav3large_lr12e-05': 1,
#  'upload_model/debertav3large_lr13e-05': 0,
#  'debertav3large_lr8e-06_att_0007': 1,
#  'debertav3large_lr9e-06_att_0007': 1,
#  'debertav3large_lr11e-05_att_0007': 1,
#  'debertav3large_lr12e-05_att_0007': 0,
#  'debertav3large_lr13e-05_att_0007': 0,
#  'debertav3large_lr14e-05_att_0007': 1,
#  'debertav3large_lr15e-05_att_0007': 0,
#  'debertav3large_lr16e-05_att_0007': 0,
#  'debertav3large_lr17e-05_att_0007': 0,
#  'debertav3large_lr18e-05_att_0007': 1,
#  'debertav3large_lr6e-06_att_0006': 1,
#  'debertav3large_lr7e-06_att_0006': 0,
#  'debertav3large_lr8e-06_att_0006': 1}


# Value:  0.47623294237768676
# Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
# {'upload_model/debertav3base_lr22e-05': 0.31611353255666413,
#  'upload_model/debertav3large_lr12e-05': 0.4865869028883723,
#  'debertav3large_lr8e-06_att_0007': 0.4262645629989505,
#  'debertav3large_lr9e-06_att_0007': 0.6016147381680528,
#  'debertav3large_lr11e-05_att_0007': 0.531486831481976,
#  'debertav3large_lr14e-05_att_0007': 0.18456428603709735,
#  'debertav3large_lr18e-05_att_0007': 0.5094108368580108,
#  'debertav3large_lr6e-06_att_0006': 0.19869967624910256,
#  'debertav3large_lr8e-06_att_0006': 0.7153757700914547}


# Value:  0.474565053465253
# {'upload_model/debertav3base_lr17e-05': 0.10605670751223599,
#  'upload_model/debertav3base_lr18e-05': 0.8151372494853444,
#  'upload_model/debertav3base_lr21e-05': -0.17229060080581526,
#  'upload_model/debertav3base_lr22e-05': 0.46124428559398767,
#  'upload_model/debertav3large_lr12e-05': 0.5213469207997752,
#  'upload_model/debertav3large_lr13e-05': 0.33587208988869377,
#  'debertav3large_lr8e-06_att_0007': 0.14044911254263082,
#  'debertav3large_lr11e-05_att_0007': 0.7504131871645143,
#  'debertav3large_lr13e-05_att_0007': 0.6190990620288409,
#  'debertav3large_lr14e-05_att_0007': -0.12134513614876408,
#  'debertav3large_lr17e-05_att_0007': 0.8917623536805175,
#  'debertav3large_lr18e-05_att_0007': -0.07388702753834085,
#  'debertav3large_lr7e-06_att_0006': 0.7887573166417163,
#  'debertav3large_lr8e-06_att_0006': 0.7195186292932556}

# Value:  0.4920279789740877

# Value:  0.4742319767376059
# {'upload_model/debertav3base_lr17e-05': -0.09377885311717854,
#  'upload_model/debertav3base_lr18e-05': 0.26760320345410316,
#  'upload_model/debertav3base_lr21e-05': 0.88539432526781,
#  'upload_model/debertav3base_lr22e-05': 0.2935469679741507,
#  'upload_model/debertav3large_lr12e-05': 0.2033256861331822,
#  'upload_model/debertav3large_lr13e-05': 0.4433115247767127,
#  'debertav3large_lr8e-06_att_0007': 0.04675646827396546,
#  'debertav3large_lr11e-05_att_0007': 0.34646024174157664,
#  'debertav3large_lr13e-05_att_0007': -0.0582651730084329,
#  'debertav3large_lr14e-05_att_0007': 0.43846159372900956,
#  'debertav3large_lr17e-05_att_0007': 0.8677427876536555,
#  'debertav3large_lr18e-05_att_0007': 0.4427276761785578,
#  'debertav3large_lr7e-06_att_0006': 0.3011481466829745,
#  'debertav3large_lr8e-06_att_0006': 0.9613027370178678,
#  'debertav3large_lr12e-05_clean_text': 1.7024715723522574,
#  'debertav3large_lr7e-06_clean_text': 1.7762890263192577,
#  'debertav3large_lr8e-06_clean_text': 1.7733730146858633}

# Value:  0.4736596085393129
# {'upload_model/debertav3base_lr17e-05': -0.42714818429335777,
#  'upload_model/debertav3base_lr18e-05': 0.6006425574784804,
#  'upload_model/debertav3base_lr21e-05': 0.2872232605158334,
#  'upload_model/debertav3base_lr22e-05': 0.42822073813160527,
#  'upload_model/debertav3large_lr12e-05': 0.5375131266654083,
#  'upload_model/debertav3large_lr13e-05': 0.53145405551725,
#  'debertav3large_lr8e-06_att_0007': -0.14203117484296998,
#  'debertav3large_lr11e-05_att_0007': 0.39667694838241024,
#  'debertav3large_lr13e-05_att_0007': 0.4038910679295646,
#  'debertav3large_lr14e-05_att_0007': 0.13930448236676002,
#  'debertav3large_lr17e-05_att_0007': 0.8530494548738371,
#  'debertav3large_lr18e-05_att_0007': -0.11474598952902004,
#  'debertav3large_lr7e-06_att_0006': 0.573636572433278,
#  'debertav3large_lr8e-06_att_0006': 0.25103546345067773,
#  'debertav3large_lr12e-05_clean_text': -0.10625543347740543,
#  'debertav3large_lr7e-06_clean_text': 0.804519691705804,
#  'debertav3large_lr8e-06_clean_text': 0.18404255780368686}

# # Value:  0.473264804390935

# Value:  0.47579687207458526
# {'upload_model/debertav3base_lr17e-05': -0.4584521019285323,
#  'upload_model/debertav3base_lr18e-05': 0.35657793888637473,
#  'upload_model/debertav3base_lr21e-05': 0.5254525428979288,
#  'upload_model/debertav3base_lr22e-05': 0.295101403704541,
#  'upload_model/debertav3large_lr12e-05': 0.5567205554812437,
#  'upload_model/debertav3large_lr13e-05': 0.9628404481437787,
#  'debertav3large_lr8e-06_att_0007': 0.9541992240007228,
#  'debertav3large_lr11e-05_att_0007': 0.7563604229334446,
#  'debertav3large_lr13e-05_att_0007': 0.4773221931525713,
#  'debertav3large_lr14e-05_att_0007': 0.1631474121193286,
#  'debertav3large_lr17e-05_att_0007': 0.560998940274175,
#  'debertav3large_lr18e-05_att_0007': 0.22442245592329782,
#  'debertav3large_lr7e-06_att_0006': 0.7308355347546078,
#  'debertav3large_lr8e-06_att_0006': 0.8432284776295845,
#  'debertav3large_lr12e-05_clean_text': 0.6105120624791835,
#  'debertav3large_lr7e-06_clean_text': 0.7784015817477296,
#  'debertav3large_lr8e-06_clean_text': 0.7255528211557266}



In [74]:
# for model in CFG.list_model_infer:
#     weight_for_model[model] = trial_.params[model]
# weight_for_model

## Predict

In [75]:
drop_columns_2 = [
                # "fold", 
                "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title", 
                "prompt_text",
                "input"
               ] + [
                f"content_pred_{i}" for i in range(CFG.n_splits)
                ] + [
                f"wording_pred_{i}" for i in range(CFG.n_splits)
                ]

In [76]:
pred_dict = {}
for target in targets:
    models = model_dict[target]
    preds = []

    for fold, model in enumerate(models):
        X_eval_cv = test.drop(columns=drop_columns_2)
        # print(X_eval_cv.head())
        pred = model.predict(X_eval_cv)
        # print('pred shape'  , pred.shape)
        preds.append(pred)
    
    pred_dict[target] = preds

KeyError: "['input', 'content_pred_0', 'content_pred_1', 'content_pred_2', 'content_pred_3', 'wording_pred_0', 'wording_pred_1', 'wording_pred_2', 'wording_pred_3'] not found in axis"

In [ ]:
for target in targets:
    preds = pred_dict[target]
    for i, pred in enumerate(preds):
        test[f"{target}_pred_{i}"] = pred

    test[target] = test[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

In [ ]:
test

,student_id,prompt_id,text,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,prompt_text,prompt_length,...,wording_pred_1,content_pred_1,wording_pred_2,content_pred_2,wording_pred_3,content_pred_3,wording_pred,content_pred,content,wording
0,000000ffffff,abc123,Example text 1,3,Example text 1,0,Summarize...,Example Title 1,Heading\nText...,3,...,-1.150856,-1.73808,-1.567721,-1.456033,-1.557598,-1.396392,-1.632939,-1.372714,-1.542456,-1.433212
1,111111eeeeee,def789,Example text 2,3,Example text 2,0,Summarize...,Example Title 2,Heading\nText...,3,...,-1.150856,-1.73808,-1.567721,-1.456033,-1.557598,-1.396392,-1.635602,-1.375756,-1.542456,-1.433212
2,222222cccccc,abc123,Example text 3,3,Example text 3,0,Summarize...,Example Title 1,Heading\nText...,3,...,-1.150856,-1.73808,-1.567721,-1.456033,-1.557598,-1.396392,-1.625720,-1.386526,-1.542456,-1.433212
3,333333dddddd,def789,Example text 4,3,Example text 4,0,Summarize...,Example Title 2,Heading\nText...,3,...,-1.150856,-1.73808,-1.567721,-1.456033,-1.557598,-1.396392,-1.626241,-1.385102,-1.542456,-1.433212


## Create Submission file

In [ ]:
sample_submission

,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


In [ ]:
test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)

## Summary

CV result is like this.

| | content rmse |wording rmse | mcrmse | LB| |
| -- | -- | -- | -- | -- | -- |
|baseline| 0.494 | 0.630 | 0.562 | 0.509 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-baseline-content-and-wording-models)|
| use title and question field | 0.476| 0.619 | 0.548 | 0.508 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-w-prompt-title-question-fields) |
| Debertav3 + LGBM | 0.451 | 0.591 | 0.521 | 0.461 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-lgbm-with-feature-engineering) |
| Debertav3 + LGBM with spell autocorrect | 0.448 | 0.581 | 0.514 | 0.459 |nogawanogawa's original code
| Debertav3 + LGBM with spell autocorrect and tuning | 0.442 | 0.566 | 0.504 | 0.453 | this notebook |

The CV values improved slightly, and the LB value is improved.